#  Silver data transformation

In [0]:
df = spark.read.format("delta")\
    .option("inferSchema","True")\
    .option("header","True")\
    .load("abfss://bronze@addscasestudy.dfs.core.windows.net/netflix_titles")

display(df)

In [0]:
df = df.fillna({"duration_minutes": "0", "duration_seasons": "1"})
df.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df = df.withColumn("duration_minutes",col('duration_minutes').cast(IntegerType()))\
        .withColumn("duration_seasons",col('duration_seasons').cast(IntegerType()))

In [0]:
df.display()

In [0]:
df = df.withColumn("short_title",split(col('title'),':')[0])
df.display()

In [0]:
df = df.withColumn("short_rating",split(col('rating'),'-')[0])
df.display()

In [0]:
df = df.withColumn("type_flag",when(col('type')=='Movie',1)\
                                .when(col('type')=='TV Show',2)\
                                .otherwise(0))
df.display()


In [0]:
from pyspark.sql.window import Window

df = df.withColumn("duration_ranking",dense_rank().over(Window.orderBy(col('duration_minutes').desc())))
df.display()


In [0]:
df.createOrReplaceTempView("Temp_view")

df = spark.sql("""
               
               select dense_rank() over (order by duration_minutes desc) as duration_ranking_sql, * 
               from Temp_view 

               """)

df.display()

In [0]:
df_vis = df.groupBy(col('type')).agg(count('*').alias('count_type'))
df_vis.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df.write.format("delta").mode("overwrite")\
    .option("path","abfss://silver@addscasestudy.dfs.core.windows.net/netflix_titles")\
    .save()